In [ ]:
# unzip data
!unzip /kaggle/input/restaurant-revenue-prediction/test.csv.zip
!unzip /kaggle/input/restaurant-revenue-prediction/train.csv.zip

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
std = StandardScaler()

# read data and drop Id column
train = pd.read_csv('./train.csv').drop('Id', axis=1)
test = pd.read_csv('./test.csv').drop('Id', axis=1)

# Feature preprocessing

In [ ]:
# check for null
train.columns[train.isna().any()].tolist(), test.columns[test.isna().any()].tolist()

There is no null values.

In [ ]:
# change datetime format
train['Open Date'] = pd.to_datetime(train['Open Date'], format='%m/%d/%Y')   
test['Open Date'] = pd.to_datetime(test['Open Date'], format='%m/%d/%Y')

# count days from opening
train['OpenDays'] = (pd.to_datetime('now') - train['Open Date']).astype('timedelta64[D]').astype(int)
test['OpenDays'] = (pd.to_datetime('now') - test['Open Date']).astype('timedelta64[D]').astype(int)

# drop Open Date column
train.drop('Open Date', axis=1)
test.drop('Open Date', axis=1)

In [ ]:
cityPerc = train[["City Group", "revenue"]].groupby(['City Group'],as_index=False).mean()
#sns.barplot(x='City Group', y='revenue', data=cityPerc)

citygroupDummy = pd.get_dummies(train['City Group'])
train = train.join(citygroupDummy)

citygroupDummyTest = pd.get_dummies(test['City Group'])
test = test.join(citygroupDummyTest)

train = train.drop('City Group', axis=1)
test = test.drop('City Group', axis=1)
test

In [ ]:
#Regression on everything
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1.1)
sns.set_style("ticks")

import numpy
xTrain = pd.DataFrame({'OpenDays':train['OpenDays'].apply(numpy.log),
                      'Big Cities':train['Big Cities'], 'Other':train['Other'],
                      'P2':train['P2'], 'P8':train['P8'], 'P22':train['P22'],
                      'P24':train['P24'], 'P28':train['P28'], 'P26':train['P26']})
#xTrain = trainData.drop(['revenue'], axis=1)
#xTrain['OpenDays'] = xTrain['OpenDays'].apply(numpy.log)
yTrain = train['revenue'].apply(numpy.log)
xTest = pd.DataFrame({'OpenDays':test['OpenDays'].apply(numpy.log),
                      'Big Cities':test['Big Cities'], 'Other':test['Other'],
                     'P2':test['P2'], 'P8':test['P8'], 'P22':test['P22'],
                      'P24':test['P24'], 'P28':test['P28'], 'P26':test['P26']})

from sklearn import linear_model

cls = RandomForestRegressor(n_estimators=150)
cls.fit(xTrain, yTrain)
pred = cls.predict(xTest)
pred = numpy.exp(pred)
cls.score(xTrain, yTrain)

In [ ]:
pred = cls.predict(xTest)
pred = numpy.exp(pred)
pre

In [ ]:
pred2 = []
for i in range(len(pred)):
    if pred[i] != float('Inf'):
        pred2.append(pred[i])

m = sum(pred2) / float(len(pred2))

for i in range(len(pred)):
    if pred[i] == float('Inf'):
        print("haha")
        pred[i] = m

In [ ]:
testData = pd.read_csv("../input/test.csv")
submission = pd.DataFrame({
        "Id": testData["Id"],
        "Prediction": pred
    })
submission.to_csv('RandomForestSimple.csv',header=True, index=False)